{DATASET_NAME} one of the datasets following the naming used in the paper:

WaterDrop
Water
Sand
Goop
MultiMaterial
RandomFloor
WaterRamps
SandRamps
FluidShake
FluidShakeBox
Continuous
WaterDrop-XL
Water-3D
Sand-3D
Goop-3D

In [ ]:
import wget

DATASET_NAME = "WaterDrop"
BASE_URL=f"https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/{DATASET_NAME}/"



for file in ["metadata.json", "train.tfrecord", "valid.tfrecord", "test.tfrecord"]:
    wget.download(f"{BASE_URL}{file}", "dataset/water_drop")

In [1]:
# https://github.com/vahidk/tfrecord
from tfrecord.torch.dataset import TFRecordDataset
from torch.utils.data import DataLoader

train_path = "dataset/water_drop/test.tfrecord"
metadata_path =  "C:/Users/p.menegatti/PycharmProjects/GNN/physics-simulation/dataset/water_drop/metadata.json"
with open(metadata_path, "r") as f:
    metadata = f.read()
index_path = None
dataset = TFRecordDataset(train_path, index_path, metadata)
print("Dataset loaded")

Dataset loaded


In [4]:
import tfrecord

loader = tfrecord.tfrecord_loader(train_path, None, metadata)
for record in loader:
    print(record)

AttributeError: 'str' object has no attribute 'items'

In [2]:
dataset[0]

NotImplementedError: 

In [3]:
loader = DataLoader(dataset, batch_size=32)

data = next(iter(loader))
print(data)

AttributeError: 'str' object has no attribute 'items'

In [3]:
import tensorflow as tf

@tf.function
def list_record_features(tfrecords_path):
    # Dict of extracted feature information
    features = {}
    # Iterate records
    for rec in tf.data.TFRecordDataset([str(tfrecords_path)]):
        # Get record bytes
        example_bytes = rec.numpy()
        # Parse example protobuf message
        example = tf.train.Example()
        example.ParseFromString(example_bytes)
        # Iterate example features
        for key, value in example.features.feature.items():
            # Kind of data in the feature
            kind = value.WhichOneof('kind')
            # Size of data in the feature
            size = len(getattr(value, kind).value)
            # Check if feature was seen before
            if key in features:
                # Check if values match, use None otherwise
                kind2, size2 = features[key]
                if kind != kind2:
                    kind = None
                if size != size2:
                    size = None
            # Save feature data
            features[key] = (kind, size)
    return features



test_path = "dataset/water_drop/test.tfrecord"

tfrecords_path = test_path
# Make some test records
with tf.io.TFRecordWriter(tfrecords_path) as writer:
    for i in range(10):
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    # Fixed length
                    'id': tf.train.Feature(
                        int64_list=tf.train.Int64List(value=[i])),
                    # Variable length
                    'data': tf.train.Feature(
                        float_list=tf.train.FloatList(value=range(i))),
                }))
        writer.write(example.SerializeToString())
# Print extracted feature information
features = list_record_features(tfrecords_path)
print(*features.items(), sep='\n')

AttributeError: in converted code:
    relative to C:\Users:

    p.menegatti\AppData\Local\Temp\ipykernel_4388\596908444.py:8 list_record_features  *
        for rec in tf.data.TFRecordDataset([str(tfrecords_path)]):
    p.menegatti\Anaconda3\envs\rdkit\lib\site-packages\tensorflow_core\python\autograph\operators\control_flow.py:648 reduce_body
        body(iterate)
    P02A0~1.MEN\AppData\Local\Temp\tmph0t7n83o.py:20 loop_body_1
        example_bytes = ag__.converted_call(rec.numpy, list_record_features_scope.callopts, (), None, list_record_features_scope)

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [5]:
def prepare_data_from_tfds(data_path='data/train.tfrecord', is_rollout=False, batch_size=2):
    import functools
    import tensorflow.compat.v1 as tf
    import tensorflow_datasets as tfds
    import reading_utils
    import tree
    from tfrecord.torch.dataset import TFRecordDataset
    def prepare_inputs(tensor_dict):
        pos = tensor_dict['position']
        pos = tf.transpose(pos, perm=[1, 0, 2])
        target_position = pos[:, -1]
        tensor_dict['position'] = pos[:, :-1]
        num_particles = tf.shape(pos)[0]
        tensor_dict['n_particles_per_example'] = num_particles[tf.newaxis]
        if 'step_context' in tensor_dict:
            tensor_dict['step_context'] = tensor_dict['step_context'][-2]
            tensor_dict['step_context'] = tensor_dict['step_context'][tf.newaxis]
        return tensor_dict, target_position
    def batch_concat(dataset, batch_size):
        windowed_ds = dataset.window(batch_size)
        initial_state = tree.map_structure(lambda spec: tf.zeros(shape=[0] + spec.shape.as_list()[1:], dtype=spec.dtype),dataset.element_spec)
        def reduce_window(initial_state, ds):
            return ds.reduce(initial_state, lambda x, y: tf.concat([x, y], axis=0))
        return windowed_ds.map(lambda *x: tree.map_structure(reduce_window, initial_state, x))
    def prepare_rollout_inputs(context, features):
        out_dict = {**context}
        pos = tf.transpose(features['position'], [1, 0, 2])
        target_position = pos[:, -1]
        out_dict['position'] = pos[:, :-1]
        out_dict['n_particles_per_example'] = [tf.shape(pos)[0]]
        if 'step_context' in features:
            out_dict['step_context'] = features['step_context']
        out_dict['is_trajectory'] = tf.constant([True], tf.bool)
        return out_dict, target_position

    metadata_path =  "C:/Users/p.menegatti/PycharmProjects/GNN/physics-simulation/dataset/water_drop/metadata.json"
    with open(metadata_path, "r") as f:
        metadata = f.read()
    ds = tf.data.TFRecordDataset([data_path])
    ds = ds.map(functools.partial(reading_utils.parse_serialized_simulation_example, metadata=metadata))
    if is_rollout:
        ds = ds.map(prepare_rollout_inputs)
    else:
        split_with_window = functools.partial(
            reading_utils.split_trajectory,
            window_length=6 + 1)
        ds = ds.flat_map(split_with_window)
        ds = ds.map(prepare_inputs)
        ds = ds.repeat()
        ds = ds.shuffle(512)
        ds = batch_concat(ds, batch_size)
    ds = tfds.as_numpy(ds)
    for i in range(100): # clear screen
        print()
    return ds
prepare_data_from_tfds()

ImportError: This version of TensorFlow Datasets requires TensorFlow version >= 2.1.0; Detected an installation of version 1.15.0. Please upgrade TensorFlow to proceed.